### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumi猫re (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
# cell for work
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [4]:
# more work
movies.movie_id.unique().shape

(35479,)

In [5]:
# and still more
reviews.shape[0]

863866

In [6]:
# and even more - open additional cells as necessary
genre_keywords = []

for genre in movies['genre']:
    try:
        genre_keywords.extend(genre.split('|'))
    except:
        continue
    
genre_keywords = set(genre_keywords)

print(genre_keywords)
len(genre_keywords)

{'Romance', 'Crime', 'Mystery', 'Talk-Show', 'Reality-TV', 'Horror', 'War', 'Adult', 'Sport', 'Game-Show', 'Sci-Fi', 'Musical', 'Film-Noir', 'Short', 'Adventure', 'Fantasy', 'Family', 'Animation', 'Documentary', 'History', 'Biography', 'Thriller', 'Action', 'Drama', 'Comedy', 'Western', 'Music', 'News'}


28

In [7]:
reviews.user_id.unique().shape

(67353,)

In [8]:
reviews['rating'].isnull().sum()

0

In [9]:
reviews.rating.describe()

count    863866.000000
mean          7.315878
std           1.853831
min           0.000000
25%           6.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: rating, dtype: float64

In [10]:
reviews['rating'].sum() / reviews['rating'].shape[0]

7.315877693994207

In [11]:
reviews['rating'].min()

0

In [12]:
reviews['rating'].max()

10

In [13]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 35479,
'The number of ratings in the dataset': 863866,
'The number of different genres': 28,
'The number of unique users in the dataset': 67353,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.315877693994207,
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [14]:
movies.head()['movie']

0        Edison Kinetoscopic Record of a Sneeze (1894)
1                  La sortie des usines Lumi猫re (1895)
2                        The Arrival of a Train (1896)
3    The Oxford and Cambridge University Boat Race ...
4                           Le manoir du diable (1896)
Name: movie, dtype: object

In [15]:
# Extract the date from the title

movies['date'] = movies['movie'].apply(lambda x: x[-5:-1] if x[-1] == ')' else np.nan)

In [16]:
# Duumy the date column for different centuries
for century in ["1800's", "1900's", "2000's"]:
    movies[century] = movies['date'].apply(lambda x: 1 if x[0:2] == century[0:2] else 0)

In [17]:
# Dummy the genre column
def dummy_genre(value):
    try:
        word_list = value.split('|')
        if genre in word_list:
            return 1
        else:
            return 0
    except:
        return 0
    
for genre in genre_keywords:
    movies[genre] = movies['genre'].apply(dummy_genre)

In [18]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [19]:
import datetime

# change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)))

In [20]:
reviews.date

0        2013-10-05 17:00:50
1        2020-04-09 17:01:12
2        2020-01-14 22:10:27
3        2020-01-09 15:50:53
4        2020-01-13 17:48:17
                 ...        
863861   2020-01-31 19:37:28
863862   2020-04-03 20:00:52
863863   2020-04-17 03:03:35
863864   2020-04-17 03:17:32
863865   2013-03-09 07:37:35
Name: date, Length: 863866, dtype: datetime64[ns]

In [21]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4  ...  month_9  month_10  month_11  \
0        0        0        0        0  ...        0         1         0   
1        0        0        0        0  ...        0         0         0   
2        0        0        0        0  ...        0         1         0   
3        0        0        0        0  ...        0         0         0   
4        0        0        0        0  ...        0         0         0   

   month_12  year_2013  year_2014  year_2015  year_2016  year_2017  year_2018  
0     